[9:50 AM] Yuliya Tarabalka
https://github.com/Lydorn/Polygonization-by-Frame-Field-Learning
GitHub - Lydorn/Polygonization-by-Frame-Field-Learning: This repository contains the code for our fast polygonal building extraction from overhead images pipeline.
This repository contains the code for our fast polygonal building extraction from overhead images pipeline. - Lydorn/Polygonization-by-Frame-Field-Learning
 
[9:54 AM] Yuliya Tarabalka
could you pls see : 1) if it would be easy to integrate this model within giQ? 2) can we run comparison of our model and this model on a few Inria benchmark images and some Pleaides neo images?
 

# Setup

In [1]:
#! mkdir git 

In [2]:
#! cd git && git clone --recursive --jobs 8 https://github.com/younes-abid/Polygonization-by-Frame-Field-Learning.git

In [3]:
#! cd git && git checkout develop

In [4]:
# conda create --name building-detection python==3.10.8 -y
# conda activate building-detection
# cd git/Polygonization-by-Frame-Field-Learning/ && pip install -r requirements.txt
# cd git/Polygonization-by-Frame-Field-Learning/ && bash setup.sh

# Download Data

https://drive.google.com/drive/folders/19yqseUsggPEwLFTBl04CmGmzCZAIOYhy?usp=sharing

https://project.inria.fr/aerialimagelabeling/



# Help

In [5]:
!pwd

/Users/younes.abid/Desktop/inference_test/train/building_detection/git/Polygonization-by-Frame-Field-Learning/notebooks


In [7]:
! cd .. && python ./main.py --help

usage: main.py [-h] [--in_filepath [IN_FILEPATH ...]]
               [--out_dirpath OUT_DIRPATH] [-c CONFIG]
               [--dataset_params DATASET_PARAMS] [-r RUNS_DIRPATH]
               [--run_name RUN_NAME] [--new_run]
               [--init_run_name INIT_RUN_NAME] [--samples SAMPLES]
               [-b BATCH_SIZE] [--eval_batch_size EVAL_BATCH_SIZE]
               [-m {train,eval,eval_coco}] [--fold [{train,val,test} ...]]
               [--max_epoch MAX_EPOCH] [--eval_patch_size EVAL_PATCH_SIZE]
               [--eval_patch_overlap EVAL_PATCH_OVERLAP]
               [--master_addr MASTER_ADDR] [--master_port MASTER_PORT] [-n N]
               [-g GPUS] [-nr NR]

options:
  -h, --help            show this help message and exit
  --in_filepath [IN_FILEPATH ...]
                        For launching prediction on several images, use this
                        argument to specify their paths.If --out_dirpath is
                        specified, prediction outputs will be saved t

# Train

An example use is for training a model with a certain config file, like so: 
python main.py --config configs/config.mapping_dataset.unet_resnet101_pretrained 

which will train the Unet-Resnet101 on the CrowdAI Mapping Challenge dataset. The batch size can be adjusted like so: 

python main.py --config configs/config.mapping_dataset.unet_resnet101_pretrained -b <new batch size>

When training is done, the script can be launched in eval mode, to evaluate the trained model: 
    
python main.py --config configs/config.mapping_dataset.unet_resnet101_pretrained --mode eval
    
Depending on the eval parameters of the config file, running this will output results on the test dataset.

Finally, if you wish to compute AP and AR metrics with the COCO API, you can run: 
    
python main.py --config configs/config.mapping_dataset.unet_resnet101_pretrained --mode eval_coco

In [8]:
! cd .. && python main.py --config configs/config.mapping_dataset.unet_resnet101_pretrained

Training on fold(s): ['train']
INFO: Loading defaults from configs/config.defaults.mapping_dataset.json
	INFO: Loading defaults from configs/config.defaults.json
			INFO: Loading defaults from configs/loss_params.json
			INFO: Loading defaults from configs/optim_params.json
			INFO: Loading defaults from configs/polygonize_params.json
		INFO: Loading defaults from configs/dataset_params.mapping_dataset.json
		INFO: Loading defaults from configs/eval_params.mapping_dataset.json
			INFO: Loading defaults from configs/eval_params.defaults.json
	INFO: Loading defaults from configs/backbone_params.unet_resnet101.json
GPU 0 -> Ready. There are 0 GPU(s) available on this node.
GPU 0 -> ERROR: Data root directory amongst "['/data/titane/user/nigirard/data/mapping_challenge_dataset', '/Users/younes.abid/data/mapping_challenge_dataset', '/data/mapping_challenge_dataset']" not found!


# Download trained model

https://drive.google.com/drive/folders/1poTQbpCz12ra22CsucF_hd_8dSQ1T3eT?usp=sharing.

In [58]:
!  cd "./git/Polygonization-by-Frame-Field-Learning/frame_field_learning/runs/mapping_dataset.unet_resnet101_pretrained.train_val _ 2020-09-07 11_28_51/checkpoints/" && \
tar -xvf checkpoint.best_val.epoch_000047.tar

tar: Error opening archive: Unrecognized archive format


In [56]:
ls "./git/Polygonization-by-Frame-Field-Learning/frame_field_learning/runs/mapping_dataset.unet_resnet101_pretrained.train_val _ 2020-09-07 11_28_51/checkpoints/"

checkpoint.best_val.epoch_000047.tar


In [10]:
# open  '/Users/younes.abid/Desktop/inference_test/train/building detection/git/Polygonization-by-Frame-Field-Learning/lydorn_utils/lydorn_utils/run_utils.py' 
#put this function in line 537
# put this on line 582
    # if run_name and "|" not in run_name:
        # run_name = reformat_file_name(run_name)
def reformat_file_name(path):
    """
    Reformats the file name to replace certain underscores with colons and a single space with a pipe in the timestamp.
    Specifically targets the format:
    'mapping_dataset.unet_resnet101_pretrained.train_val _ 2020-09-07 11_28_51'
    to transform it into:
    'mapping_dataset.unet_resnet101_pretrained.train_val | 2020-09-07 11:28:51'
    """
    import os
    dir_path, file_name = os.path.split(path)
    
    # Identify the base and date parts based on the expected format ending with ' _ YYYY-MM-DD HH_MM_SS'
    if ' _ ' in file_name:
        # Split at the specific ' _ ' sequence that precedes the date
        base_part, date_time_part = file_name.split(' _ ', 1)

        # Replace underscores in the date-time part with colons where they represent time
        date_part, time_part = date_time_part.split(' ')
        time_part = time_part.replace('_', ':', 2)

        # Assemble the new file name with a properly trimmed pipe symbol
        file_name = f"{base_part.strip()} | {date_part} {time_part}"

    return os.path.join(dir_path, file_name)


In [11]:
# fix this function this way
def setup_run_dir(runs_dirpath, run_name=None, new_run=False, check_exists=False):
    """
    If new_run is True, creates a new directory:
        If run_name is None, generate a random name
        else build the created directory name with run_name

    If new_run is False, return an existing directory:
        if run_name is None, return the last created directory (from timestamp)
        else return the last created directory (from timestamp) whose name starts with run_name,
        if that does not exist and check_exists is False create a new run with run_name,
        if check_exists is True, then raise an error.

    Special case: if there is no existing runs, the new_run is not taken into account and the function behaves like new_run is True.

    :param runs_dirpath: Parent directory path of all the runs
    :param run_name:
    :param new_run:
    :param check_exists:
    :return: Run directory path. The directory name is in the form "run_name | timestamp"
    """
    #B_FIX
    # if run_name and "|" not in run_name:
    #     run_name = reformat_file_name(run_name)
    #E_FIX
    # Create runs directory of it does not exist
    if not os.path.exists(runs_dirpath):
        os.makedirs(runs_dirpath, exist_ok=True)

    existing_run_dirnames = os.listdir(runs_dirpath)
    print(existing_run_dirnames)
    if new_run or (not new_run and not 0 < len(existing_run_dirnames)):
        if run_name is not None:
            # Create another directory name for the run, with its name starting with run_name
            name_timestamped = create_name_timestamped(run_name)
        else:
            # Create another directory name for the run, excluding the existing names
            existing_run_names = [existing_run_dirname.split(" | ")[0] for existing_run_dirname in
                                  existing_run_dirnames]
            name_timestamped = create_free_name_timestamped(exclude_list=existing_run_names)
        current_run_dirpath = os.path.join(runs_dirpath, name_timestamped)
        os.mkdir(current_run_dirpath)
    else:
        if run_name is not None:
            # Pick run dir based on run_name
            #B_FIX
            filtered_existing_run_dirnames = [existing_run_dirname for existing_run_dirname in existing_run_dirnames if
                                              existing_run_dirname.split(" _ ")[0] == run_name]
            
            # filtered_existing_run_dirnames = [existing_run_dirname for existing_run_dirname in existing_run_dirnames if
            #                                   existing_run_dirname.split(" | ")[0] == run_name]
            #E_FIX
            if filtered_existing_run_dirnames:
                #B_FIX
                filtered_existing_run_timestamps = [filtered_existing_run_dirname.split(" _ ")[1] for
                                                    filtered_existing_run_dirname in
                                                    filtered_existing_run_dirnames]
                # filtered_existing_run_timestamps = [filtered_existing_run_dirname.split(" | ")[1] for
                #                                     filtered_existing_run_dirname in
                #                                     filtered_existing_run_dirnames]
                #E_FIX
                filtered_last_index = filtered_existing_run_timestamps.index(max(filtered_existing_run_timestamps))
                current_run_dirname = filtered_existing_run_dirnames[filtered_last_index]
            else:
                if check_exists:
                    raise FileNotFoundError("Run '{}' does not exist.".format(run_name))
                else:
                    return setup_run_dir(runs_dirpath, run_name=run_name, new_run=True)
        else:
            # Pick last run dir based on timestamp
            #B_FIX
            existing_run_timestamps = [existing_run_dirname.split(" _ ")[1] for existing_run_dirname in
                                       existing_run_dirnames]
            # existing_run_timestamps = [existing_run_dirname.split(" | ")[1] for existing_run_dirname in
            #                            existing_run_dirnames]
            #E_FIX
            last_index = existing_run_timestamps.index(max(existing_run_timestamps))
            current_run_dirname = existing_run_dirnames[last_index]
        current_run_dirpath = os.path.join(runs_dirpath, current_run_dirname)
    return current_run_dirpath


In [12]:
path = "/Users/younes.abid/Desktop/inference_test/train/building_detection/git/Polygonization-by-Frame-Field-Learning/frame_field_learning/runs/mapping_dataset.unet_resnet101_pretrained.train_val _ 2020-09-07 11_28_51"
path = "mapping_dataset.unet_resnet101_pretrained.train_val _ 2020-09-07 11_28_51"
reformat_file_name(path)

'mapping_dataset.unet_resnet101_pretrained.train_val | 2020-09-07 11:28:51'

In [13]:
ls "/Users/younes.abid/Desktop/inference_test/train/building detection/git/Polygonization-by-Frame-Field-Learning/data/building detection model/data/mapping_challenge_dataset/raw/val/images/000000000000.jpg"

ls: /Users/younes.abid/Desktop/inference_test/train/building detection/git/Polygonization-by-Frame-Field-Learning/data/building detection model/data/mapping_challenge_dataset/raw/val/images/000000000000.jpg: No such file or directory


# Predict

In [14]:
ls -l "/Users/younes.abid/Desktop/inference_test/train/building detection/git/Polygonization-by-Frame-Field-Learning/frame_field_learning/runs/"

ls: /Users/younes.abid/Desktop/inference_test/train/building detection/git/Polygonization-by-Frame-Field-Learning/frame_field_learning/runs/: No such file or directory


In [14]:
ls '/Users/younes.abid/Desktop/inference_test/train/building detection/git/Polygonization-by-Frame-Field-Learning/frame_field_learning/runs/mapping_dataset.unet_resnet101_pretrained.train_val _ 2020-09-07 11_28_51'

args.json    checkpoints/ config.json


In [15]:
! cd .. && python main.py \
--runs_dirpath "/Users/younes.abid/Desktop/inference_test/train/building_detection/git/Polygonization-by-Frame-Field-Learning/frame_field_learning/runs/" \
--run_name "mapping_dataset.unet_resnet101_pretrained.train_val" \
--in_filepath "/Users/younes.abid/Desktop/inference_test/train/building_detection/git/Polygonization-by-Frame-Field-Learning/data/building detection model/data/mapping_challenge_dataset/raw/val/images/000000000000.jpg"

ERROR: Run 'mapping_dataset.unet_resnet101_pretrained.train_val' does not exist.


In [1]:
import rasterio
import osgeo

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
pip install GDAL==3.6.2

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip list |grep rasterio

rasterio                  1.3.10
Note: you may need to restart the kernel to use updated packages.


In [108]:
! conda install rasterio==1.3.10 -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /Users/younes.abid/opt/anaconda3/envs/building-detection

  added / updated specs:
    - rasterio==1.3.10


The following NEW packages will be INSTALLED:

  affine             pkgs/main/noarch::affine-2.3.0-pyhd3eb1b0_0 
  attrs              pkgs/main/osx-64::attrs-23.1.0-py310hecd8cb5_0 
  blas               pkgs/main/osx-64::blas-1.0-mkl 
  blosc              pkgs/main/osx-64::blosc-1.21.3-hcec6c5f_0 
  boost-cpp          pkgs/main/osx-64::boost-cpp-1.82.0-ha357a0b_2 
  c-ares             pkgs/main/osx-64::c-ares-1.19.1-h6c40b1e_0 
  cairo              pkgs/main/osx-64::cairo-1.16.0-h3ce6f7e_5 
  certifi            pkgs/main/osx-64::certifi-2024.7.4-py310hecd8cb5_0 
  cfitsio            pkgs/main/osx-64::cfitsio-3.470-hbd21b

In [15]:
JVM_MEMORY_BASE = 10  # in GB
ADDITIONAL_TOOL_MEMORY = 10  # in GB
MAX_MEMORY = 40 # in GB

def calculate_total_memory(image_size_1, image_size_2, AOI_percentage):
    total_zip_files_size = image_size_1 + image_size_2
    JVM_memory_work = round(total_zip_files_size) * AOI_percentage * 5
    JVM_memory = JVM_MEMORY_BASE + JVM_memory_work
    total_memory = min(JVM_memory + ADDITIONAL_TOOL_MEMORY, MAX_MEMORY)
    return total_memory

In [22]:
calculate_total_memory(0.4, 0.5, 0.5)

22.5

In [21]:
calculate_total_memory(0.8, 0.5, 1.0)

25.0

In [25]:
import re

log_entry = """
docker-compose-inference-worker-1 | [INFO] 2024-08-28 08:20:26,362: [inferenceworker.worker] -> [run_inference]: younes 247 ---------------------------------------------------------------------------------------------------- 
job=Job(id='1') request=RequestDetectionStrict(resolution_default=0.5, resolution_minimum=0.1, resolution_maximum=5.0, file_url='/data/video_frames/tmp35fdz0wi/frame_150.jpg', is_geotiff=False, aoi=None, confidence_threshold=0.25, nms_threshold=0.45) model=ModelVideoObjectDetection(resolution_default=0.5, resolution_minimum=0.1, resolution_maximum=5.0, activation='None', id='fish_recognition_latest', version='1', date='2024-08-26', description='YOLOv5 object detector and classifier trained on fish data', labels=['fish'], family='yolov5', type=<ModelType.video: 'FULL_MOTION_VIDEO'>, inputs=[ModelInput(dtype=<Dtypes.FLOAT32: 'float32'>, shape=(3, 640, 640))], preprocessing_filters=[], job_type='video_object_detection', tile_size=640, channel_first=True, padding=16, preprocessing_mask=None, postprocessing_filter=None, efficientdet_config={}, confidence_threshold=0.25, nms_threshold=0.45, classifier=ModelClassification(resolution_default=0.5, resolution_minimum=0.1, resolution_maximum=5.0, activation='None', id='fish_classifier_latest', version='1', date='2024-08-26', description='EfficientNet classifier for fish species', labels=['Bristlenose_catfish', 'Gold', 'Guppy', 'Jacks', 'Paradise Fish', 'Red_tail_black_shark', 'Sea Horse Fish', 'angel_fish', 'bala', 'barracuda', 'butterfly_fish', 'cichild', 'clown', 'crab', 'damsel_fishes', 'dolphin', 'eel', 'groupers', 'hatchetfish', 'jellyfish', 'king_fish', 'lion', 'luderick', 'neon tetra', 'octopus', 'penguins', 'porkfish', 'puffer', 'sail_fish', 'sea cow', 'seal', 'shark', 'shrimp', 'snappers', 'squid', 'starfish', 'stingray', 'surgeon_fish', 'sword_fish', 'tuna', 'turtle_tortoise', 'unknown', 'wrasse', 'zebra', 'zebra loach'], family='classification', type=<ModelType.video: 'FULL_MOTION_VIDEO'>, inputs=[ModelInput(dtype=<Dtypes.FLOAT32: 'float32'>, shape=(3, 128, 128))], preprocessing_filters=[], job_type='classification', tile_size=128, channel_first=True, padding=16, preprocessing_mask=None, postprocessing_filter=None, efficientdet_config={}, confidence_threshold=0.1, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
"""

# Simple regex to match and extract log message
pattern = r"(?<=\-\-\>).*"
matches = re.findall(pattern, log_entry, re.DOTALL)

for match in matches:
    print("Log Message:")
    print(match.strip())

In [26]:
matches

[]